In [59]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib  # Librería estándar para guardar modelos y scalers


Información

- Obtener los medicamentos mas relevantes
- Intentar obtener lugar_residencia, lugar_procedencia, destino_alta y vive_solo --> No tenemos datos
- Ver como hago la columna gdiagalt --> De momento lo dejaria con los códigos, no hay una clasificación clara.


### Carga de archivos

In [60]:
df = pd.read_csv('data/data_2018_01_01-2026_01_13/DatosIdentificativosAdministrativos_combinado.csv')
df_glucemias = pd.read_csv('data/data_2018_01_01-2026_01_13/GlucemiasGlucosurias_combinado.csv')
df_antecedentes = pd.read_csv('data/data_2018_01_01-2026_01_13/ValoracionesAntecedentesPersonales_combinado.csv')
df_valoracion_enfermeria = pd.read_csv('data/data_2018_01_01-2026_01_13/ValoracionesEnfermeria_combinado.csv')
df_prescripcion_farmacos = pd.read_csv('data/data_2018_01_01-2026_01_13/PrescripcionFarmacos_combinado.csv')
df_valoracion_medica = pd.read_csv('data/data_2018_01_01-2026_01_13/ValoracionesMedicas_combinado.csv')
df_constantes = pd.read_csv('data/data_2018_01_01-2026_01_13/ConstantesVitales_combinado.csv')

#df_accidentes = pd.read_csv('../datos/AccidentesPlanta.csv')
#df_dietas = pd.read_csv('../datos/DietasViasAlimentacion.csv')
#df_infecciones = pd.read_csv('../datos/InfeccionesNosocomiales.csv')
#df_ulceras = pd.read_csv('data_2025_06_01-2026_01_11/UlcerasPresion_combinado.csv')
#df_prestaciones =  pd.read_csv('data_2025_06_01-2026_01_11/Prestaciones_combinado.csv')

/tmp/ipykernel_1892/1391586130.py:5: DtypeWarning: Columns (57) have mixed types. Specify dtype option on import or set low_memory=False.
  df_prescripcion_farmacos = pd.read_csv('data/data_2018_01_01-2026_01_13/PrescripcionFarmacos_combinado.csv')
/tmp/ipykernel_1892/1391586130.py:7: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_constantes = pd.read_csv('data/data_2018_01_01-2026_01_13/ConstantesVitales_combinado.csv')


### Datos identificaticos

In [61]:
# Eliminar las columnas que no tienen información relevante
df = df.drop(columns=['gprovinc', 'dnomprov', 'gpoblaci','dpoblaci','odiagini', 'odiagalt','dnomprov','ganoadme','gnumadme','gmotingr','dmotingr','gdiaging','gdiagini', 'gserulti', 'dnomserv','dsitalta','dmotalta','itraslad','odiaging','gdiasec1', 'odiasec1', 'gdiasec2', 'odiasec2'])

In [62]:
# Reemplazar 'H' por 0 y 'M' por 1 en la columna 'itipsexo'
df['itipsexo'] = df['itipsexo'].replace({'H': 0, 'M': 1})


# Columna ds_izq_der para saber que lado de la cadera es 
# ------------------------------------------------------------------------------------------
# Listas de códigos para "DERECHO" y "IZQUIERDO"
derecho_codes = ["M66.151","M80.051A","M97.01XA","M97.01XD", "M97.01XS","S72.001A","S72.011A","S72.091A","S72.101A","S72.141A","S73.031A","S79.811A", "T84.010","T84.010A","T84.010D","T84.010S","T84.040","T84.040A","T84.040D","T84.040S", "T84.090A"]
izquierdo_codes = ["M66.152","M97.02XA","M97.02XD","M97.02XS", "S70.02XA","S72.002A","S72.012A","S72.102A","S72.112A","S72.142A","S72.142D","S72.22XA","T84.011", "T84.011A","T84.011D","T84.011S","T84.021A","T84.031A","T84.041", "T84.041A","T84.041D","T84.041S",]
no_especificado = ["M66.15","M66.159","M84.359","M84.359A","M84.359D","M84.359G","M84.359K","M84.359P","M84.359S","M84.459","M84.459A","M84.459D","M84.459G","M84.459K","M84.459P","M84.459S","M84.559","M84.559A","M84.559D","M84.559G","M84.559K",
                    "M84.559P","M84.559S","M84.659","M84.659A","M84.659D","M84.659G","M84.659K","M84.659P","M84.659S","T84.84XA","Z51.89"]
# Categorizar la columna
def categorize_side(value):
    value = str(value)  # Convertir el valor a string por seguridad
    if any(code in value for code in no_especificado):
        return 0
    elif any(code in value for code in izquierdo_codes):
        return 1
    elif any(code in value for code in derecho_codes):
        return 2
    else:
        return None  # Marcar para eliminar

# Crear la nueva columna 'ds_izq_der' basada en la función de categorización
df['ds_izq_der'] = df['gdiagalt'].apply(categorize_side)
df['ds_izq_der'] = pd.to_numeric(df['ds_izq_der'], errors='coerce').fillna(-999).astype(int)


# Eliminar las filas donde 'ds_izq_der' es None 
df = df[(df['ds_izq_der'] != -999)]

# ------------------------------------------------------------------------------------------

# Columna turnos de trabajo
# ------------------------------------------------------------------------------------------
# Convertir la columna 'fllegada' a formato de fecha y hora
df['fllegada'] = pd.to_datetime(df['fllegada'])

# Función para asignar el turno en función de la hora
def asignar_turno(hora):
    if 8 <= hora < 15:      #Mañana
        return 0
    elif 15 <= hora < 22:   # Tarde
        return 1
    else:                   # Noche
        return 2

# Crear una nueva columna 'ds_turno' basada en la hora de 'fllegada'
df['ds_turno'] = df['fllegada'].dt.hour.apply(asignar_turno)
# ------------------------------------------------------------------------------------------

# Columna edad
# ------------------------------------------------------------------------------------------
# Convertir la columna 'fnacipac' a formato de fecha
df['fnacipac'] = pd.to_datetime(df['fnacipac'])

# Función para calcular la edad
def calcular_edad(fecha_nacimiento):
    hoy = datetime.now()
    edad = hoy.year - fecha_nacimiento.year - ((hoy.month, hoy.day) < (fecha_nacimiento.month, fecha_nacimiento.day))
    return edad

# Crear la nueva columna 'ds_edad' calculando la edad para cada fila
df['ds_edad'] = df['fnacipac'].apply(calcular_edad)
df = df[df['ds_edad'] >= 50] # Filtrar pacientes mayores de 50 años, menores hay muy pocos y estropea la muestra
# ------------------------------------------------------------------------------------------

# Columnas de días estancia, preopertorio y postoperatorio
# ------------------------------------------------------------------------------------------
# Convertir las columnas de fecha a formato datetime, sin considerar las horas
df['fllegada'] = pd.to_datetime(df['fllegada']).dt.normalize()
df['faltplan'] = pd.to_datetime(df['faltplan']).dt.normalize()
df['finterve'] = pd.to_datetime(df['finterve']).dt.normalize()

# Calcular los días de estancia total (faltplan - fllegada)
df['ds_estancia'] = (df['faltplan'] - df['fllegada']).dt.days
df = df[df['ds_estancia'] <= 50] # Eliminar estancias mayores a 50 días, que son outliers

# Calcular los días de preoperatorio incluyendo el día de intervención
df['ds_pre_oper'] = (df['finterve'] - df['fllegada']).dt.days
df['ds_pre_oper'] = pd.to_numeric(df['ds_pre_oper'], errors='coerce').fillna(-999).astype(int)
#df = df[df['ds_pre_oper'] <= 20] # Eliminar estancias mayores a 20 días, que son outliers


# Calcular los días de postoperatorio incluyendo el día de intervención
df['ds_post_oper'] = (df['faltplan'] - df['finterve']).dt.days
df['ds_post_oper'] = pd.to_numeric(df['ds_post_oper'], errors='coerce').fillna(-999).astype(int)

# ------------------------------------------------------------------------------------------
# Eliminar filas donde 'gsitalta' es NaN (vacia) antes de calcular si vive o no
df.dropna(subset=['gsitalta'], inplace=True)
df['gsitalta'] = df['gsitalta'].astype(int)
df = df[df['gsitalta'] != 0]


# Crear la columna 'ds_vivo_alta' basada en la condición de 'gsitalta'
df['ds_vivo_alta'] = df['gsitalta'].apply(lambda x: 0 if x == 7 else 1)

# Reemplazar 'N' por 0 y 'S' por 1 en la columna 'ireingre'
df['ireingre'] = df['ireingre'].replace({'N': 0, 'S': 1})

# Crear la columna 'ds_dia_semana_llegada'(1 para lunes, 7 para domingo)
df['ds_dia_semana_llegada'] = df['fllegada'].dt.dayofweek + 1

# Crear la columna 'ds_mes_llegada'  
df['ds_mes_llegada'] = df['fllegada'].dt.month

# Obtenemos si viven en el centro o en las afueras
codigos_centro = [24001, 24002, 24003, 24004, 24005, 24006, 24007, 24008, 24009, 24010, 24012, 24070, 24071, 24080, ]  #Códigos del centro
df['ds_centro_afueras'] = df['gcodipos'].apply(lambda x: 1 if x in codigos_centro else 0)

# Reemplazar 'I' por 0 y 'U' por 1 en la columna 'itipingr'
df['itipingr'] = df['itipingr'].replace({'I': 0, 'U': 1})

# Reemplazar 'N' por 0 y 'S' por 1 en la columna 'iotrocen'
df['iotrocen'] = df['iotrocen'].replace({'N': 0, 'S': 1})

# Eliminar pacientes que no tienen nada que ver con Fractura de cadera
# Lista de valores que quieres eliminar
valores_a_eliminar = ['FDFD192912290', 'FRGR193404470', 'GNMR193911510']

# Eliminar filas donde gidenpac tenga valores en la lista
df = df[~df['gidenpac'].isin(valores_a_eliminar)]

# Eliminar columnas que ya no necesitamos
#df = df.drop(columns=['faltplan'])
#df = df.drop(columns=['fnacipac'])
#df = df.drop(columns=['fllegada'])
#df = df.drop(columns=['finterve'])

/tmp/ipykernel_1892/3855050951.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['itipsexo'] = df['itipsexo'].replace({'H': 0, 'M': 1})
/tmp/ipykernel_1892/3855050951.py:100: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['ireingre'] = df['ireingre'].replace({'N': 0, 'S': 1})
/tmp/ipykernel_1892/3855050951.py:116: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behav

### Accidentes


In [63]:
# No tiene info que podamos usar
"""
# Filtrar df_accidentes para seleccionar solo las columnas relevantes
df_accidentes = df_accidentes[['gidenpac', 'ifamilia', 'gmovilid']]

# Realizar el merge con df, añadiendo las columnas de 'ifamilia' donde coincida 'gidenpac'
df = pd.merge(df, df_accidentes, on='gidenpac', how='left')

# Reemplazar 'N' por 0 y 'S' por 1 en la columna 'ifamilia'
df['ifamilia'] = df['ifamilia'].replace({'N': 0, 'S': 1})

# Cambiar el nombre de la columna 'ifamilia' a 'vive_solo'
df = df.rename(columns={'ifamilia': 'vive_solo'})

# Renombrar 'gmovilid' como 'movilidad_accidentes'
df = df.rename(columns={'gmovilid': 'movilidad_accidentes'})

# Eliminar filas duplicadas basándose en la columna 'gidenpac'
df = df.drop_duplicates(subset='gidenpac')
# Pasar a entero y añadimos -999 en los vacios
#df['vive_solo'] = df['vive_solo'].fillna(-999).astype(int)
#df['movilidad_accidentes'] = df['movilidad_accidentes'].fillna(-999).astype(int)
"""

"\n# Filtrar df_accidentes para seleccionar solo las columnas relevantes\ndf_accidentes = df_accidentes[['gidenpac', 'ifamilia', 'gmovilid']]\n\n# Realizar el merge con df, añadiendo las columnas de 'ifamilia' donde coincida 'gidenpac'\ndf = pd.merge(df, df_accidentes, on='gidenpac', how='left')\n\n# Reemplazar 'N' por 0 y 'S' por 1 en la columna 'ifamilia'\ndf['ifamilia'] = df['ifamilia'].replace({'N': 0, 'S': 1})\n\n# Cambiar el nombre de la columna 'ifamilia' a 'vive_solo'\ndf = df.rename(columns={'ifamilia': 'vive_solo'})\n\n# Renombrar 'gmovilid' como 'movilidad_accidentes'\ndf = df.rename(columns={'gmovilid': 'movilidad_accidentes'})\n\n# Eliminar filas duplicadas basándose en la columna 'gidenpac'\ndf = df.drop_duplicates(subset='gidenpac')\n# Pasar a entero y añadimos -999 en los vacios\n#df['vive_solo'] = df['vive_solo'].fillna(-999).astype(int)\n#df['movilidad_accidentes'] = df['movilidad_accidentes'].fillna(-999).astype(int)\n"

### Constantes vitales

In [64]:
# Convertir fechas una sola vez
df['fllegada'] = pd.to_datetime(df['fllegada'], errors='coerce')
df_constantes['fapuntes'] = pd.to_datetime(df_constantes['fapuntes'], errors='coerce')

# Función optimizada
def get_closest_vitals(group, df_const):
    """Para cada paciente, encuentra el registro más cercano"""
    gidenpac = group.name
    fllegada = group['fllegada'].iloc[0]
    
    # Filtrar solo registros de este paciente
    registros = df_const[df_const['gidenpac'] == gidenpac].copy()
    if registros.empty:
        return pd.Series({'ntensmin': -999, 'ntensmax': -999, 'ntempera': -999.0, 'nsatuoxi': -999})
    
    # Calcular distancia
    registros['diff'] = (registros['fapuntes'] - fllegada).abs()
    mas_cercano = registros.loc[registros['diff'].idxmin()]
    
    return pd.Series({
        'ntensmin': mas_cercano['ntensmin'],
        'ntensmax': mas_cercano['ntensmax'],
        'ntempera': mas_cercano['ntempera'],
        'nsatuoxi': mas_cercano['nsatuoxi']
    })

# Aplicar por paciente (mucho más rápido)
vitals_cercanas = df.groupby('gidenpac').apply(lambda g: get_closest_vitals(g, df_constantes))
df = df.join(vitals_cercanas, on='gidenpac')

# Conversiones
df['ntensmin'] = pd.to_numeric(df['ntensmin'], errors='coerce').fillna(-999).astype(int)
df['ntensmax'] = pd.to_numeric(df['ntensmax'], errors='coerce').fillna(-999).astype(int)
df['ntempera'] = pd.to_numeric(df['ntempera'], errors='coerce').fillna(-999).astype(float)
df['nsatuoxi'] = pd.to_numeric(df['nsatuoxi'], errors='coerce').fillna(-999).astype(int)

# Filtros
df = df[(df['ntensmin'] >= 30) & (df['ntensmin'] <= 105)]
df = df[(df['ntensmax'] >= 40) & (df['ntensmax'] <= 200)]
df = df[df['nsatuoxi'] >= 80]

/tmp/ipykernel_1892/910770467.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  vitals_cercanas = df.groupby('gidenpac').apply(lambda g: get_closest_vitals(g, df_constantes))


### Antecedentes


In [65]:
# Seleccionar solo las columnas necesarias de 'df_antecedentes' y crear una copia
df_ant = df_antecedentes[['gidenpac', 'dconclin', 'vbivalor']].copy()

# Crear columnas en 'df_ant' basadas en el contenido de 'dconclin'
df_ant['ds_HTA'] = df_ant["dconclin"].str.contains("HTA", na=False).astype(int)
df_ant['ds_alergia_medicamentosa'] = df_ant["dconclin"].str.contains("Alergia medicamentosa").astype(int)
df_ant['ds_alergia_alimenticia'] = df_ant["dconclin"].str.contains("Alergia alimenticia").astype(int)
df_ant['ds_diabetes'] = df_ant["dconclin"].str.contains("Diabetes Mellitus").astype(int)
df_ant['ds_otras_alergias'] = df_ant["dconclin"].str.contains("Otras alergias").astype(int)

# Actualizar valores en columnas de antecedentes usando 'vbivalor' si no es nulo
columns_to_update = ['ds_HTA', 'ds_alergia_medicamentosa', 'ds_alergia_alimenticia', 'ds_otras_alergias']
for col in columns_to_update:
    df_ant[col] = df_ant.apply(lambda x: x['vbivalor'] if pd.notnull(x['vbivalor']) and x[col] == 1 else x[col], axis=1)

# Reemplazar valores específicos ('N' -> 0, 'S' -> 1, 'A' -> 1) en todo el DataFrame
df_ant.replace({'N': 0, 'S': 1, 'A': 1}, inplace=True)

# Eliminar las columnas 'dconclin' y 'vbivalor' si ya no son necesarias
df_ant.drop(columns=['dconclin', 'vbivalor'], inplace=True)

# Eliminar duplicados en el DataFrame (si aún no se ha hecho)
df_ant = df_ant.drop_duplicates()

# Agrupar por 'gidenpac' y usar 'max' para que cualquier 1 en una columna mantenga el valor como 1
df_ant = df_ant.groupby('gidenpac', as_index=False).max()

# Realizar un merge entre df y df_ant sin sufijos
df = pd.merge(df, df_ant, on='gidenpac', how='left')

# Asignar valores de df_ant solo si son 1
for col in columns_to_update:
    df[col] = df.apply(lambda x: x[col] if pd.notnull(x[col]) and x[col] == 1 else np.nan if pd.isna(x[col]) else x[col], axis=1)


/tmp/ipykernel_1892/2776989086.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_ant.replace({'N': 0, 'S': 1, 'A': 1}, inplace=True)


### Valoracion Enfermeria

In [66]:
# Seleccionar solo las columnas necesarias de 'df_valoracion_enfermeria'
df_valoracion_enfermeria = df_valoracion_enfermeria[['gidenpac', 'dconclin', 'ncodtabu', 'nsecuval', 'nvalncon', 'ovallcon', 'vbivalor']].copy()

# Asegurarse de que 'ncodtabu', 'nvalncon', 'ovallcon', y 'vbivalor' sean numéricos
df_valoracion_enfermeria[['ncodtabu', 'nvalncon', 'ovallcon', 'vbivalor']] = df_valoracion_enfermeria[['ncodtabu', 'nvalncon', 'ovallcon', 'vbivalor']].apply(pd.to_numeric, errors='coerce')

# Definir los valores de 'dconclin' que se convertirán en nuevas columnas
patterns = {
    '_movilidad': "_Movilidad",
    'movilidad': "Movilidad",
    'lugar_residencia': "G Lugar de Residencia",
    'lugar_procedencia': "G Procedencia",
    'destino_alta': "G Destino Alta",
    'Barthel': "Resultado Indice de Barthel",
    'barthel_alta': "Resultado Indice de Barthel al Alta",
    'braden': "Resultado Escala de Braden",
    'riesgo_caida': "Resultado Escala Riesgo Caidas"
}

# Crear una expresión regular unida para filtrar las filas deseadas
regex_pattern = "|".join(patterns.values())

# Filtrar el DataFrame para conservar solo las filas que coinciden con el patrón
df_valoracion_enfermeria = df_valoracion_enfermeria[df_valoracion_enfermeria['dconclin'].str.contains(regex_pattern, na=False)]

# Iterar sobre el diccionario para crear las columnas basadas en el valor máximo entre las columnas especificadas
for new_col, pattern in patterns.items():
    # Crear la nueva columna con el valor máximo de las columnas seleccionadas donde 'dconclin' coincide con el patrón
    df_valoracion_enfermeria[new_col] = np.where(
        df_valoracion_enfermeria['dconclin'].str.contains(pattern, na=False),
        df_valoracion_enfermeria[['ncodtabu', 'nvalncon', 'ovallcon', 'vbivalor']].max(axis=1),
        np.nan
    )

# Agrupar por 'gidenpac' y mantener el valor máximo en cada columna para eliminar duplicados
df_valoracion_enfermeria = df_valoracion_enfermeria.groupby('gidenpac', as_index=False).max()

# Eliminar las columnas no deseadas del DataFrame
df_valoracion_enfermeria = df_valoracion_enfermeria.drop(columns=['dconclin', 'ncodtabu', 'nsecuval', 'nvalncon', 'ovallcon', 'vbivalor', '_movilidad','barthel_alta'])

# Combinar df y df_valoracion_enfermeria por la columna 'gidenpac'
df= pd.merge(df, df_valoracion_enfermeria, on='gidenpac', how='left')

# Convertimos a enteros y los Nan los pasamos a -999
df['movilidad'] = df['movilidad'].fillna(-999).astype(int)
df = df[df['movilidad'] != -999] # Mantenemos solo los registros donde movilidad NO sea -999
df = df[df['movilidad'] <= 4] # Eliminamos registros con movilidad mayor a 4, que no tienen sentido

df['lugar_residencia'] = df['lugar_residencia'].fillna(-999).astype(int)
df['lugar_procedencia'] = df['lugar_procedencia'].fillna(-999).astype(int)
df['destino_alta'] = df['destino_alta'].fillna(-999).astype(int)
df['Barthel'] = df['Barthel'].fillna(-999).astype(int)
df = df[df['Barthel'] != -999] # Mantenemos solo los registros que no sean -999
df['braden'] = df['braden'].fillna(-999).astype(int)
df = df[df['braden'] != -999] # Mantenemos solo los registros que no sean -999
df['riesgo_caida'] = df['riesgo_caida'].fillna(-999).astype(int)
df = df[df['riesgo_caida'] != -999] # Mantenemos solo los registros donde riesgo_caida NO sea -999


### Glucemias

In [67]:
# Pocos datos

# Crear un conjunto con los valores de gidenpac en df_glucemias
glucemias_pacientes = set(df_glucemias['gidenpac'].dropna())

# Crear la columna 'ds_diabetes' solo en las casillas vacías. Añadimos info a la casilla ds_diabetes
df['ds_diabetes'] = df.apply(lambda row: 1 if pd.isna(row['ds_diabetes']) and row['gidenpac'] in glucemias_pacientes 
                             else (0 if pd.isna(row['ds_diabetes']) else row['ds_diabetes']), axis=1)


### Valoraciones Medicas

In [68]:
# 2. Definir los códigos
codigos = [1368, 1369, 1370, 1371, 1372, 1373, 1374, 1742, 1743, 1750, 1813, 1814, 1815]

# 3. Filtrar directamente sobre el dataframe ya cargado
df_valoracion_medica = df_valoracion_medica[df_valoracion_medica['gcodconc'].isin(codigos)].copy()

# 4. Seleccionar columnas
df_valoracion_medica = df_valoracion_medica[['gidenpac', 'gcodconc', 'ovallcon']]

# 5. Guardar
#df_valoracion_medica.to_csv('DF_VAL_MEDICAS_COMBINED.csv', index=False)

# 6. Conteo
conteo_codigos = df_valoracion_medica['ovallcon'].value_counts()
print(conteo_codigos)

ovallcon
LOS PREVIOS                                           2087
HTA                                                   1165
FRACTURA SUBCAPITAL DE CADERA DERECHA                  879
FRACTURA PERTROCANTEREA DE CADERA DERECHA              872
DISLIPEMIA                                             480
                                                      ... 
FRACTURA SUBCAPITAL DE FÉMUR DERECHO FEMUR DERECHO       1
ÚLCERA POR PRESIÓN ESTADIO IV                            1
FRACTURA DE LA CADERA IZQ                                1
S.IQUAW+PIE EQUINO IZDO                                  1
DIABETES MELLITUS TIPO2                                  1
Name: count, Length: 8131, dtype: int64


In [69]:
# Crear la nueva columna 'ds_HTA' para detectar 'HTA' en cualquier parte del texto
df_valoracion_medica['ds_HTA'] = df_valoracion_medica['ovallcon'].str.contains('HTA', case=False, na=False).astype(int)
df_valoracion_medica['ds_ITU'] = df_valoracion_medica['ovallcon'].str.contains('ITU', case=False, na=False).astype(int)
df_valoracion_medica['ds_anemia'] = df_valoracion_medica['ovallcon'].str.contains('ANEMIA', case=False, na=False).astype(int)
df_valoracion_medica['ds_vitamina_d'] = df_valoracion_medica['ovallcon'].str.contains('VITAMINA D', case=False, na=False).astype(int)
df_valoracion_medica['ds_obesidad'] = df_valoracion_medica['ovallcon'].str.contains('OBESIDAD', case=False, na=False).astype(int)
df_valoracion_medica['ds_osteoporosis'] = df_valoracion_medica['ovallcon'].str.contains('OSTEOPOROSIS', case=False, na=False).astype(int)
df_valoracion_medica['ds_acido_folico'] = df_valoracion_medica['ovallcon'].str.contains('ACIDO FOLICO', case=False, na=False).astype(int)


# Insuficiencia respiratoria
# ___________________________________________________________
respiratory_terms = [
    'INSUFICIENCIA RESPIRATORIA AGUDA', 'INSUFICIENCIA RESPIRATORIA', 
    'INSUFICIENCIA RESPIRATORIA AGUDA PARCIAL', 'INSUFICIENCIA RESPIRATORIA CRÓNICA AGUDIZADA', 
    'INSUFICIENCIA RESPIRATORIA PARCIAL', 'INSUFICIENCIA RESPIRATORIA 2RIA', 
    'INSUFICIENCIA RESPIRATORIA GLOBAL', 'INSUFICIENCIA RESPIRATORIA AGUDA SECUNDARIA', 
    'INSUFICIENCIA RESPIRATORIA PARCIAL CRÓNICA AGUDIZADA', 
    'INSUFICIENCIA RESPIRATORIA AGUDA PARCIAL RESUELTA', 'EPOC AGUDIZADO', 'EPOC', 'EPOC REAGUDIZADO'
]

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(respiratory_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_valoracion_medica['ds_insuficiencia_respiratoria'] = df_valoracion_medica['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Diabetes
# ___________________________________________________________
diabetes_terms = [
    'DM TIPO 2 ', 'DM TIPO II', 'DIABETES MELLITUS', 'DM-II',  'DIABETES MELLITUS TIPO II','DIABETES'
]

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(diabetes_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_valoracion_medica['ds_diabetes'] = df_valoracion_medica['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Infección respiratoria
# ___________________________________________________________
infeccion_respiratoria_terms = [
   'INFECCIÓN RESPIRATORIA', 'INFECCIÓN RESPIRATORIA DE VÍAS BAJAS',  'INFECCIÓN RESPIRATORIA DE VÍAS INFERIORES', 'INFECCION RESPIRATORIA', 'INFECCION RESPIRATORIA AGUDA', 'INFECCIÓN RESPIRATORIA AGUDA', 'INFECCIÓN RESPIRATORIA DE VÍAS BAJAS NO CONDENSANTE'
]

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(infeccion_respiratoria_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
#df_val_medicas_combined['ds_infeccion_respiratoria'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Insuficiencia Cardiaca
# ___________________________________________________________
insuficiencia_cardiaca_terms = [
'INSUFICIENCIA CARDIACA', 'INSUFICIENCIA CARDIACA DESCOMPENSADA',  'INSUFICIENCIA RESPIRATORIA CRÓNICA', 'INSUFICIENCIA CARDIACA CONGESTIVA',  'I. CARDIACA', 'INSUFICIENCIA CARDIACA CONGESTIVA DESCOMPENSADA', 'INSUFICIENCIA CARDÍACA', 'INSUFICIENCIA CARDIACA CRÓNICA DESCOMPENSADA',  'INSUFICIENCIA CARDIACA CONGESTIVA AGUDA', 'ICC AGUDA', 'CARDIOPATIA ISQUEMICA',  'CARDIOPATÍA ISQUÉMICA', 'ICC', 'ICC DESCOMPENSADA']

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(insuficiencia_cardiaca_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_valoracion_medica['ds_insuficiencia_cardiaca'] = df_valoracion_medica['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Deterioro cognitivo
# ___________________________________________________________
deterioro_cognitivo_terms = [
	'DETERIORO COGNITIVO', 'DETERIORO COGNITIVO', 'DEMENCIA']

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(deterioro_cognitivo_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_valoracion_medica['ds_deterioro_cognitivo'] = df_valoracion_medica['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Exitus
# ___________________________________________________________
exitus_terms = [ 'EXITUS', 'ÉXITUS']

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(exitus_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_valoracion_medica['ds_exitus'] = df_valoracion_medica['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Insuficiencia renal
# ___________________________________________________________
insuficiencia_renal_terms = [
    'I. RENAL', 'INSUFICIENCIA RENAL', 'ENFERMEDAD RENAL CRÓNICA', 'INSUFICIENCIA RENAL AGUDA', 'INSUFICIENCIA RENAL CRÓNICA AGUDIZADA', 'INSUFICIENCIA RENAL CRONICA', 'IRC', 'ERC REAGUDIZADA', 'ERC AGUDIZADA', 'ERC']

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(insuficiencia_renal_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_valoracion_medica['ds_insuficiencia_renal'] = df_valoracion_medica['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________


# Eliminamos las columnas que ya no necesitamos
df_filtrado = df_valoracion_medica.drop(columns=['gcodconc', 'ovallcon'])

# Agrupar por 'gidenpac' y aplicar max() para cada columna
df_val_medicas_procesado = df_filtrado.groupby('gidenpac').max().reset_index()

df_val_medicas_procesado.to_csv('DF_VAL_MEDICAS_PROCESED.csv', index=False)



### Filtrado de los datos finales

Unir el DataFrame principal (df) con el de valoraciones medicas ya filtrado y procesado (df_val_medicas_procesado)

In [70]:
# Unir df y df_val_medicas_procesado por la columna 'gidenpac'
df = df.merge(df_val_medicas_procesado, on='gidenpac', how='left')
# Reemplazar valores NaN o nulos por -999
#df.fillna(-999, inplace=True)

# Los valores vacios los imputamos al valor común, en mas de un 99% de los casos que es 0.0, sin enfermedad. Otra estrategia seria eliminarlos, pero de momento al tener pocos datos, lo dejamos así.
# Lista de las columnas de diagnósticos mostradas en la imagen
cols_diagnosticos = [
    'ds_ITU', 
    'ds_anemia', 
    'ds_vitamina_d', 
    'ds_insuficiencia_respiratoria',  # Asumo el nombre completo
    'ds_insuficiencia_cardiaca', 
    'ds_deterioro_cognitivo', 
    'ds_insuficiencia_renal',   
    'ds_obesidad',
    'ds_osteoporosis',
    'ds_acido_folico'
]

# Verificar que las columnas existan antes de intentar rellenarlas (para evitar errores)
cols_existentes = [col for col in cols_diagnosticos if col in df.columns]

# Rellenar los valores vacíos (NaN) con 0.0 en esas columnas
df[cols_existentes] = df[cols_existentes].fillna(0.0)

# Convertimos a enteros y los Nan los pasamos a -999
"""
df['ds_vitamina_d'] = df['ds_vitamina_d'].fillna(-999).astype(int)
df['ds_obesidad'] = df['ds_obesidad'].fillna(-999).astype(int)
df['ds_osteoporosis'] = df['ds_osteoporosis'].fillna(-999).astype(int)
df['ds_insuficiencia_respiratoria'] = df['ds_insuficiencia_respiratoria'].fillna(-999).astype(int)
df['ds_diabetes_y'] = df['ds_diabetes_y'].fillna(-999).astype(int)
df['ds_infeccion_respiratoria'] = df['ds_infeccion_respiratoria'].fillna(-999).astype(int)
df['ds_insuficiencia_cardiaca'] = df['ds_insuficiencia_cardiaca'].fillna(-999).astype(int)
df['ds_deterioro_cognitivo'] = df['ds_deterioro_cognitivo'].fillna(-999).astype(int)
df['ds_insuficiencia_renal'] = df['ds_insuficiencia_renal'].fillna(-999).astype(int)
df['ds_HTA_y'] = df['ds_HTA_y'].fillna(-999).astype(int)
df['ds_ITU'] = df['ds_ITU'].fillna(-999).astype(int)
df['ds_acido_folico'] = df['ds_acido_folico'].fillna(-999).astype(int)
df['ds_anemia'] = df['ds_anemia'].fillna(-999).astype(int)
"""

"\ndf['ds_vitamina_d'] = df['ds_vitamina_d'].fillna(-999).astype(int)\ndf['ds_obesidad'] = df['ds_obesidad'].fillna(-999).astype(int)\ndf['ds_osteoporosis'] = df['ds_osteoporosis'].fillna(-999).astype(int)\ndf['ds_insuficiencia_respiratoria'] = df['ds_insuficiencia_respiratoria'].fillna(-999).astype(int)\ndf['ds_diabetes_y'] = df['ds_diabetes_y'].fillna(-999).astype(int)\ndf['ds_infeccion_respiratoria'] = df['ds_infeccion_respiratoria'].fillna(-999).astype(int)\ndf['ds_insuficiencia_cardiaca'] = df['ds_insuficiencia_cardiaca'].fillna(-999).astype(int)\ndf['ds_deterioro_cognitivo'] = df['ds_deterioro_cognitivo'].fillna(-999).astype(int)\ndf['ds_insuficiencia_renal'] = df['ds_insuficiencia_renal'].fillna(-999).astype(int)\ndf['ds_HTA_y'] = df['ds_HTA_y'].fillna(-999).astype(int)\ndf['ds_ITU'] = df['ds_ITU'].fillna(-999).astype(int)\ndf['ds_acido_folico'] = df['ds_acido_folico'].fillna(-999).astype(int)\ndf['ds_anemia'] = df['ds_anemia'].fillna(-999).astype(int)\n"

Vamos a combinar las 2 columnas de HTA.
- HTA: Tenemos 2. X e Y. 
	- X viene de antecedentes
	- Y de valoraciones médicas
- Podemos combinar las 2, ya que es más probable que no pongan la info, a poner una info falsa


In [71]:
df['ds_HTA'] = df[['ds_HTA_x', 'ds_HTA_y']].max(axis=1)

# Eliminar las columnas originales
df.drop(columns=['ds_HTA_x', 'ds_HTA_y'], inplace=True)

# Igual con diabetes
df['ds_diabetes'] = df[['ds_diabetes_x', 'ds_diabetes_y']].max(axis=1)

# Eliminar las columnas originales
df.drop(columns=['ds_diabetes_x', 'ds_diabetes_y'], inplace=True)


Eliminamos filas que les falten datos importantes

In [72]:
# Eliminamos filas que no tengan los dias de pre o post operacion
df = df[df['ds_pre_oper'] != -999]  # Eliminamos filas que no tengan los dias de pre o post operacion
df = df[df['ds_post_oper'] != -999]
"""
# Si las valoraciones médicas no tienen info sobre el paciente, lo eliminamos
columnas_a_verificar = [
    'ds_ITU', 'ds_acido_folico', 'ds_anemia', 'ds_vitamina_d', 
    'ds_obesidad', 'ds_osteoporosis', 'ds_insuficiencia_respiratoria', 
    'ds_infeccion_respiratoria', 'ds_insuficiencia_cardiaca', 
    'ds_deterioro_cognitivo', 'ds_insuficiencia_renal'
]

df = df[~df[columnas_a_verificar].isin([-999]).any(axis=1)]
"""

"\n# Si las valoraciones médicas no tienen info sobre el paciente, lo eliminamos\ncolumnas_a_verificar = [\n    'ds_ITU', 'ds_acido_folico', 'ds_anemia', 'ds_vitamina_d', \n    'ds_obesidad', 'ds_osteoporosis', 'ds_insuficiencia_respiratoria', \n    'ds_infeccion_respiratoria', 'ds_insuficiencia_cardiaca', \n    'ds_deterioro_cognitivo', 'ds_insuficiencia_renal'\n]\n\ndf = df[~df[columnas_a_verificar].isin([-999]).any(axis=1)]\n"

### Prescripción Farmacos

In [73]:
# Medicamentos prescitos

# Seleccionar solo las columnas 'gidenpac' y 'tarasist'
df_prescripcion_farmacos = df_prescripcion_farmacos[['gidenpac', 'tarasist']]

# Agrupar por 'gidenpac' y combinar los medicamentos en una lista separada por comas
df_medicamentos = df_prescripcion_farmacos.groupby('gidenpac')['tarasist'].apply(lambda x: ', '.join(x.dropna().astype(str))).reset_index()

# Guardar
df_medicamentos.to_csv('medicamentos.csv', index=False)

# Procesado final para entrenar

In [74]:
# Convertir a categorico
columnas_a_convertir = ['itipsexo','ireingre','gsitalta', 'gmotalta','gdiagalt','ds_izq_der','ds_turno','ds_vivo_alta','ds_dia_semana_llegada', 'ds_mes_llegada', 'ds_HTA','ds_alergia_medicamentosa','ds_alergia_alimenticia','ds_diabetes','ds_otras_alergias', 'lugar_residencia','lugar_procedencia','destino_alta','ds_ITU','ds_anemia','ds_vitamina_d','ds_insuficiencia_respiratoria','ds_insuficiencia_cardiaca','ds_deterioro_cognitivo','ds_insuficiencia_renal' ]
df[columnas_a_convertir] = df[columnas_a_convertir].astype('category')

One-Hot Encoding

In [75]:
cols = ["gdiagalt","ds_izq_der","ds_dia_semana_llegada", "ds_mes_llegada", "ds_turno"]

cols_onehot = [c for c in cols
               if c in df.columns]

df = pd.get_dummies(df, columns=cols_onehot, drop_first=False, dtype="int8")

Eliminar columnas que no tienen informacion suficiente para utilizar

In [76]:
# Definimos la lista de columnas a eliminar o no necesitamos
cols_to_drop = ['gidenpac','ds_exitus', 'gcodipos', 'itipingr', 'ireingre', 'gmotalta', 'ds_vivo_alta','lugar_residencia','lugar_procedencia','destino_alta','ds_obesidad','ds_acido_folico',
                'faltplan','fnacipac','fllegada','finterve'
]

df.drop(columns=cols_to_drop, inplace=True, errors='ignore')

# Eliminamos columnas de gdiagalt que no aportan info porque solo tienen 1 registro
'''gdiagalt_to_drop = ['G45.9', 'G72.81', 'I69.152', 'I69.254', 'M51.26', 'M80.051A', 'M97.01XA', 'M97.02XA', 'S72.111A','S72.142D','S72.21XA','S72.24XA','S73.031A', 'T84.010A', 
                    'T84.011A','T84.030A', 'T84.051A', 'Z47.81'
]'''
gdiagalt_to_drop = []

# Añadimos el prefijo 'gdiagalt_' a cada elemento
full_cols_to_drop = ['gdiagalt_' + code for code in gdiagalt_to_drop]

# Ahora sí borramos las columnas completas
df.drop(columns=full_cols_to_drop, inplace=True, errors='ignore')

Agrupamos categorias de gsitalta

In [77]:
def agrupar_situacion_alta(valor):
    if valor in [1, 2, 9]:
        return 2
    elif valor in [3, 4, 6,7]:
        return 4
    else:
        return 99 # Categoría de seguridad por si escapa algún valor

df['gsitalta_agrupada'] = df['gsitalta'].apply(agrupar_situacion_alta)

'''
1: Curación total
2: Mejoria
3: Sin cambios
4: Agravamiento
6: Con secuelas
7: Exitus
9: Mejoria a residencia
'''

'\n1: Curación total\n2: Mejoria\n3: Sin cambios\n4: Agravamiento\n6: Con secuelas\n7: Exitus\n9: Mejoria a residencia\n'

In [78]:
# Convertimos las columnas numéricas a su tipo adecuado
# Lista de columnas a convertir
columnas_categoricas = ['itipsexo', 'gsitalta', 'gsitalta_agrupada','iotrocen','ds_centro_afueras',]

# Bucle para convertir cada columna
for col in columnas_categoricas:
    df[col] = df[col].astype('category')



Escalar los valores numéricos

In [79]:
# --- ESCALADO Y GUARDADO --- Mejor hacerlo en el pipeline de entrenamiento
'''
# 1. Escalar X (Predictoras)
cols_numericas = ['ds_edad','ds_estancia', 'ds_pre_oper','ds_post_oper', 'ntensmin', 'ntensmax', 'ntempera', 'nsatuoxi', 'Barthel', 'braden']

scaler = StandardScaler()
# Sobrescribimos en X los valores transformados
df[cols_numericas] = scaler.fit_transform(df[cols_numericas])

# 4. Guardar el Scaler (necesario para desescalar resultados futuros)
joblib.dump(scaler, 'scaler_completo.pkl') #(Para preprocesar nuevos pacientes cuando el modelo de resultados, ya que si se los damos escalados, el los devuelve escalados y debemos desescalar para ver los días)
'''

"\n# 1. Escalar X (Predictoras)\ncols_numericas = ['ds_edad','ds_estancia', 'ds_pre_oper','ds_post_oper', 'ntensmin', 'ntensmax', 'ntempera', 'nsatuoxi', 'Barthel', 'braden']\n\nscaler = StandardScaler()\n# Sobrescribimos en X los valores transformados\ndf[cols_numericas] = scaler.fit_transform(df[cols_numericas])\n\n# 4. Guardar el Scaler (necesario para desescalar resultados futuros)\njoblib.dump(scaler, 'scaler_completo.pkl') #(Para preprocesar nuevos pacientes cuando el modelo de resultados, ya que si se los damos escalados, el los devuelve escalados y debemos desescalar para ver los días)\n"

In [80]:
#
'''
# Cargar el scaler
scaler = joblib.load('scaler_completo.pkl')

# Índice de 'ds_estancia' en tu lista original (es el índice 1)
idx_estancia = 1 
media = scaler.mean_[idx_estancia]
desviacion = scaler.scale_[idx_estancia]

# Desescalar manual (Valor Real = (Valor Escalado * Desviación) + Media)
prediccion_dias_reales = (y_pred * desviacion) + media
'''

"\n# Cargar el scaler\nscaler = joblib.load('scaler_completo.pkl')\n\n# Índice de 'ds_estancia' en tu lista original (es el índice 1)\nidx_estancia = 1 \nmedia = scaler.mean_[idx_estancia]\ndesviacion = scaler.scale_[idx_estancia]\n\n# Desescalar manual (Valor Real = (Valor Escalado * Desviación) + Media)\nprediccion_dias_reales = (y_pred * desviacion) + media\n"

Guardado final

In [81]:
df.to_csv('DATOS_PREPROCESADOS.csv', index=False)